In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import sqlite3
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
# 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
driver = webdriver.Chrome('C:/Users/noilkwon/dev/기타/chromedriver.exe', chrome_options=options)

In [3]:
# DB 생성
conn = sqlite3.connect('auction.db')
cur = conn.cursor()

In [4]:
# 테이블 생성
#cur.execute('create table 경매(사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수)')
#cur.execute('drop table 경매')
cur.execute('delete from 경매')

In [5]:
# 홈페이지 접속
url = r'http://www.courtauction.co.kr/maemul/search_total.asp'
driver.get(url)

In [6]:
# 매각기일
selector = ['table#Table1','tbody','tr:nth-of-type(2)','td','#Table4','tbody','tr','td:nth-of-type(2)','input#sdate']
selector = ' > '.join(selector)
driver.find_element_by_css_selector(selector).clear()
driver.find_element_by_css_selector(selector).send_keys('2000-01-01')

In [7]:
# 소재지
select = Select(driver.find_element_by_id('sido'))
select.select_by_visible_text('서울')

In [8]:
# 용도 복수선택
driver.find_element_by_id('Checkbox8').click()

In [9]:
# 정렬방식
select = Select(driver.find_element_by_id('sort'))
select.select_by_value('1')

In [10]:
# 버튼 클릭
driver.find_element_by_id('search').click()

In [11]:
# 함수 정의
def getText(row,col):
    selector = ['#Table22','tbody','tr:nth-of-type('+str(row)+')','td:nth-of-type('+str(col)+')']
    selector = ' > '.join(selector)
    text = driver.find_element_by_css_selector(selector).text
    return text

def getPage():
    data = []
    for r in np.arange(6,45,2):
        values = []
        # 사건번호, 소재지, 가격, 매각기일, 진행상태, 조회수
        for c in [2,4,5,6,7,8]:   
            values.append(getText(r,c))
        data.append(values)
    return data

In [12]:
# 작업 시작
page = 730

for i in range(page):
    j = i//10
    driver.execute_script("GoPage("+str(j)+","+str(i+1)+",'board')")
    cur.executemany("insert into 경매 values (?,?,?,?,?,?)", getPage())
    if i%10 == 0:
        conn.commit()
        print("{}번째 페이지 적재".format(i+1))
        
conn.commit()

1번째 페이지 적재
11번째 페이지 적재
21번째 페이지 적재
31번째 페이지 적재
41번째 페이지 적재
51번째 페이지 적재
61번째 페이지 적재
71번째 페이지 적재
81번째 페이지 적재
91번째 페이지 적재
101번째 페이지 적재
111번째 페이지 적재
121번째 페이지 적재
131번째 페이지 적재
141번째 페이지 적재
151번째 페이지 적재
161번째 페이지 적재
171번째 페이지 적재
181번째 페이지 적재
191번째 페이지 적재
201번째 페이지 적재
211번째 페이지 적재
221번째 페이지 적재
231번째 페이지 적재
241번째 페이지 적재
251번째 페이지 적재
261번째 페이지 적재
271번째 페이지 적재
281번째 페이지 적재
291번째 페이지 적재
301번째 페이지 적재
311번째 페이지 적재
321번째 페이지 적재
331번째 페이지 적재
341번째 페이지 적재
351번째 페이지 적재
361번째 페이지 적재
371번째 페이지 적재
381번째 페이지 적재
391번째 페이지 적재
401번째 페이지 적재
411번째 페이지 적재
421번째 페이지 적재
431번째 페이지 적재
441번째 페이지 적재
451번째 페이지 적재
461번째 페이지 적재
471번째 페이지 적재
481번째 페이지 적재
491번째 페이지 적재
501번째 페이지 적재
511번째 페이지 적재
521번째 페이지 적재
531번째 페이지 적재
541번째 페이지 적재
551번째 페이지 적재
561번째 페이지 적재
571번째 페이지 적재
581번째 페이지 적재
591번째 페이지 적재
601번째 페이지 적재
611번째 페이지 적재
621번째 페이지 적재
631번째 페이지 적재
641번째 페이지 적재
651번째 페이지 적재
661번째 페이지 적재
671번째 페이지 적재
681번째 페이지 적재
691번째 페이지 적재
701번째 페이지 적재
711번째 페이지 적재
721번째 페이지 적재


In [68]:
# 적재 데이터 출력
cur.execute("select * from 경매")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['번호', '소재지', '가격', '매각기일', '진행상태', '조회수'])

In [69]:
# 중복 데이터 제거
u, indices = np.unique(df['번호'], return_index=True)
indices.sort()
df = df.iloc[indices,:]

In [70]:
# 번호 분리
df['법원'] = df['번호'].str.split('\n').str.get(0)
df['사건번호'] = df['번호'].str.split('\n').str.get(1)
df['용도'] = df['번호'].str.split('\n').str.get(2)

In [71]:
# 가격 분리 
df['감정가'] = df['가격'].str.split('\n').str.get(0)
df['최저입찰가'] = df['가격'].str.split('\n').str.get(1)
df['최저입찰가율'] = df['가격'].str.split('\n').str.get(2)

# 형(Type) 변환
df['최저입찰가'] = df['최저입찰가'].str.replace(',','').astype(float)
df['감정가'] = df['감정가'].str.replace(',','').astype(float)
df['최저입찰가율'] = df['최저입찰가율'].str.replace('%','').astype(float)

In [72]:
# 매각기일 추출
df['매각일'] = df['매각기일'].str.split('\n').str.get(0)

In [73]:
# 소재지 추출
df['주소'] = df['소재지'].str.split('\n').str.get(0)
df['면적'] = df['소재지'].str.split('\n').str.get(1)
df['시'] = df['주소'].str.split(' ').str.get(0)
df['구'] = df['주소'].str.split(' ').str.get(1)
df['동'] = df['주소'].str.split(' ').str.get(2)

In [74]:
# 기타정보 추출
# 건물이랑 토지 바뀌어서 들어가는 거 처리해야 함
# 문자에서 숫자만 추출해야 함
df['건물'] = df['면적'].str.split(' ').str.get(0)
df['토지'] = df['면적'].str.split(' ').str.get(1)
df['키1'] = df['면적'].str.split(' ').str.get(2)
df['키2'] = df['면적'].str.split(' ').str.get(3)
df['임시1'] = df['면적'].str.split(' ').str.get(4)
df['임시2'] = df['면적'].str.split(' ').str.get(5)
df.loc[df['임시1'].notnull(),['키1','키2']] = df.loc[df['임시1'].notnull(),['임시1','임시2']]
df.loc[df['키1']=='','키1'] = np.nan
df.loc[df['키2']=='','키2'] = np.nan

# 오류 수정
df.loc[df['사건번호']=='2012-2689[1]','토지'] = df.loc[df['사건번호']=='2012-2689[1]','건물']
df.loc[df['사건번호']=='2011-20696[2]','토지'] = df.loc[df['사건번호']=='2012-2689[1]','건물']
df.loc[df['사건번호']=='2012-2689[1]','토지'] = np.nan
df.loc[df['사건번호']=='2012-2689[1]','토지'] = np.nan
df['건물'] = df['건물'].str.extract('건물(\d+.?\d?)평', expand=False).astype('float')
df['토지'] = df['토지'].str.extract('토지(\d+.?\d?)평', expand=False).astype('float')

In [75]:
# 진행상태 추출
df['진행'] = df['진행상태'].str.split('\n').str.get(0)
df['유찰횟수'] = df['진행상태'].str.split('\n').str.get(1).str.extract('유찰(\d)회', expand=True)
df.loc[(df['유찰횟수'].isna()) & (df['진행']=='진행'),'유찰횟수'] = 0

In [77]:
df = df[['법원','사건번호','용도','감정가','최저입찰가','매각일','주소','시','구','동','건물','토지','키1','키2','진행','유찰횟수']]
df.head()

,법원,사건번호,용도,감정가,최저입찰가,매각일,주소,건물,토지,키1,키2,진행,유찰횟수
0,서울중앙지법6계,2016-9088[1],근린상가,"80,000,000.0","13,422,000.0",2018-06-27,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 2층 F2014...",1.1,0.3,NaN,NaN,진행,8
1,서울중앙지법6계,2016-12916[1],근린상가,"43,000,000.0","7,214,000.0",2018-06-27,"서울 중구 을지로6가 18-21 ,-220,-221 굿모닝시티쇼핑몰 7층 F7189...",1.1,0.3,NaN,NaN,진행,8
2,서울중앙지법6계,2017-10412[1],근린상가,"300,000,000.0","192,000,000.0",2018-06-27,서울 서초구 양재동 275-2 윈드스톤호피스텔 지하1층 148호 [논현로 79],29.4,6.4,NaN,NaN,진행,2
3,서울중앙지법6계,2017-105389[1],근린상가,"171,163,400.0","136,931,000.0",2018-06-27,서울 중구 산림동 90-4 외1 [청계천로 168],1.1,1.8,NaN,NaN,진행,1
4,서울남부지법10계,2017-1124[1],근린상가,"146,000,000.0","38,274,000.0",2018-06-19,서울 구로구 구로동 3-25 신도림테크노마트 3층 289호 외1 [새말로 97],6.2,1.4,NaN,NaN,진행,6


In [79]:
writer = pd.ExcelWriter('auction.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()